If using colab, install necessary libraries

In [1]:
! pip install urlextract

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import requests     # 2.18.4
import json         # 2.0.9
import pandas as pd # 0.23.0
import numpy as np
import ast
import time
import re
from urlextract import URLExtract as extract
extractor = extract()

In [17]:
# BFE Abnahme
#packages = 'https://ckan.ogdch-abnahme.clients.liip.ch/api/3/action/package_search?fq=organization:(bundesamt-fur-energie-bfe)&rows=500'

# BFE PROD
packages = 'https://ckan.opendata.swiss/api/3/action/package_search?fq=organization:(bundesamt-fur-energie-bfe)&rows=500'

# Make the HTTP request
response = requests.get(packages)

# Use the json module to load CKAN's response into a dictionary
response_dict = json.loads(response.content)

# Check the contents of the response
assert response_dict['success'] is True  # make sure if response is OK

# Get a list of all publications and their information (each publication is a dictionary)
data = response_dict['result']['results'].copy()

# get all upper keys
allKeys = []

# store information about each dataset in a frame
df = pd.DataFrame(data)[['id','title','url', 'relations', 'resources']]
df

,id,title,url,relations,resources
0,df6ecca9-1097-46ad-aa71-97f3f1a32f51,{'fr': 'Stations de recharge pour voitures éle...,https://www.ich-tanke-strom.ch,[{'url': 'https://github.com/SFOE/ichtankestro...,"[{'byte_size': 0, 'cache_last_updated': None, ..."
1,6729f43f-ffbd-4224-81bf-d0a7d945fcff,{'fr': 'energiedashboard.ch : Prévisions de co...,,{},"[{'byte_size': 0, 'cache_last_updated': None, ..."
2,6f77c60b-fd9a-44ac-914f-be3ccd4db5af,{'fr': 'energiedashboard.ch: Flux quotidiens d...,https://energiedashboard.ch,"[{'url': 'www.entsog.eu', 'label': 'ENTSO-G'},...","[{'byte_size': 0, 'cache_last_updated': None, ..."
3,63c37114-3f81-4419-9dd6-3c08a5e4140e,{'fr': 'energiedashboard.ch: Prix de l'énergie...,https://energiedashboard.ch,[{'url': 'https://www.epexspot.com/en/market-d...,"[{'byte_size': 0, 'cache_last_updated': None, ..."
4,2bb595ad-10f7-49a4-aeda-1e23aa07925d,{'fr': 'energiedashboard.ch: Approvisionnement...,https://energiedashboard.ch,[{'url': 'https://www.bwl.admin.ch/bwl/de/home...,"[{'byte_size': 0, 'cache_last_updated': None, ..."
...,...,...,...,...,...
96,4fbb2aee-19c5-471d-9113-ae41c6cb6adf,{'fr': 'Analyse ex-post de la consommation éne...,http://www.bfe.admin.ch/ogd15,{},"[{'byte_size': 0, 'cache_last_updated': None, ..."
97,912f238a-7783-42b3-bd90-24432751af90,{'fr': 'Analyse ex-post de la consommation éne...,http://www.bfe.admin.ch/ogd16,{},"[{'byte_size': 0, 'cache_last_updated': None, ..."
98,954b6e0b-eb35-4f3c-87f5-96e055c04b31,{'fr': 'Statistique de la consommation d’énerg...,http://www.bfe.admin.ch/ogd14,{},"[{'byte_size': 0, 'cache_last_updated': None, ..."
99,bc38ed81-1356-4721-8da0-e5cc613efe4f,{'fr': 'Statistique de la consommation d’élect...,http://www.bfe.admin.ch/ogd9,{},"[{'byte_size': 0, 'cache_last_updated': None, ..."


---------------------------------------------------------------------------

Functions

In [18]:
def check_url(url):
    try:
        time.sleep(1)
        # head or get --> head is not working consistantly
        
        # redirect not working on every machine and not everytime... but this link technically works
        if url == 'https://bfe.admin.ch/shared-mobility-angebote':
            return True
        
        get = requests.get(url)
        
        if get.status_code < 400:
            return True
        else:
            return False
        
    except requests.exceptions.RequestException as e:
        print(e)
        raise SystemExit(f"{url}: is NOT reachable \nErr: {e}")

In [19]:
def addHTTPS(url):
    adjusted = url.replace("http:", "https:" )
    adjusted = adjusted.replace("www", "https://www" )
    
    return adjusted.replace("https://https://www", "https://www" )

In [20]:
def addHTTP(url):
    adjusted = url.replace("www", "http://www")
    return adjusted.replace("http://http://www", "http://www")

In [21]:
def getURLs(original_string, urls):
    
    links = []

    badLinks       = []

    adjustedString = '' 
    for s in original_string:
      adjustedString = adjustedString + s


    needsToBeListed = False

    
    for url in urls:
      
      url_modified    = ''
      # only store links which are faulty or need a protocol update (https)
      
      
      # if https already used
      if 'https' in url:
        working = check_url(url)
        
        if not working:
          badLinks.append([url, '-'])
          #needsToBeListed = True
          
      # if https not in link
      else:
        
        url_modified = addHTTPS(url)
        working = check_url(url_modified)
        
        if working:
          needsToBeListed = True
          # replace in adjusted_string
          adjustedString = adjustedString.replace(url, url_modified)
          
        else:
          # does not change http: urls
          url_modified = addHTTP(url)
          working = check_url(url)
          
          if not working:
            badLinks.append([url, '-'])
                    
        
    if needsToBeListed:
      links.append([original_string, adjustedString])

    if len(badLinks) != 0:
      links = links + badLinks

    return links

In [22]:
def checkString(string):
  
  urls = extractor.find_urls(string)
  
  # first remove elements which are not actual links
  urls = [url for url in urls if ('http' in url) or ('www' in url)]

  # if the same url appears multiple times
  urls = list(set(urls))
  
  return getURLs(string, urls)

In [ ]:
urls = ['http://www.google.ch', 'https://www.google.ch', 'www.google.ch', 'https://data.geo.admin.ch/ch.bfe.windenergie-geschwindigkeit_h150/windenergie-geschwindigkeit_h150/windenergie-geschwindigkeit_h150_2056.fgdb.zip']
test = getURLs(str(urls), urls)

In [ ]:
for i in test:
  print(i)

["['http://www.google.ch', 'https://www.google.ch', 'www.google.ch', 'https://data.geo.admin.ch/ch.bfe.windenergie-geschwindigkeit_h150/windenergie-geschwindigkeit_h150/windenergie-geschwindigkeit_h150_2056.fgdb.zip']", "['https://https://www.google.ch', 'https://https://www.google.ch', 'https://www.google.ch', 'https://data.geo.admin.ch/ch.bfe.windenergie-geschwindigkeit_h150/windenergie-geschwindigkeit_h150/windenergie-geschwindigkeit_h150_2056.fgdb.zip']"]
['https://data.geo.admin.ch/ch.bfe.windenergie-geschwindigkeit_h150/windenergie-geschwindigkeit_h150/windenergie-geschwindigkeit_h150_2056.fgdb.zip', '-']


In [ ]:
check_url('https://www.bfe.admin.ch/waerme-kaeltenachfrage-wohnen-und-dl')

True

------------------------------------------------------------------------------------------------------------------------------------------------------

For _url_ and _relations_

In [23]:
cols = ['url', 'relations']

completeList = []

for index, row in df.iterrows():
    
    print(index)
    dataset_name = row['title']['de']
    id           = row['id']
    
    for col in cols:
  
        att_type = 'dataset'
        att      = col
        val      = row[col]

        liste = []

        if col == 'resources':
            
            att_type = 'resources'
            #change att
            print('nichts')


        else:
            if type(val) == str:
                liste = checkString(val)
            else:
                liste = checkString(str(val))


        if len(liste) == 0:
            continue
        else:
            for elem in liste:
                completeList.append([dataset_name, att_type, att, id, elem[0], elem[1]])


0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100


In [24]:
f = pd.DataFrame(completeList)
f.columns=['title', 'type', 'attribute', 'id', 'old_link', 'new_link']
f

,title,type,attribute,id,old_link,new_link
0,Ladestationen für Elektroautos,dataset,relations,df6ecca9-1097-46ad-aa71-97f3f1a32f51,https://github.com/SFOE/DIEMO-Documentation/bl...,-
1,energiedashboard.ch: Tägliche Flüsse in die un...,dataset,relations,6f77c60b-fd9a-44ac-914f-be3ccd4db5af,"[{'url': 'www.entsog.eu', 'label': 'ENTSO-G'},...","[{'url': 'https://www.entsog.eu', 'label': 'EN..."
2,energiedashboard.ch: Stromproduktion Swissgrid,dataset,relations,7676b090-6dff-483f-9d0c-166a0060b9b3,"[{'url': 'www.swissgrid.ch', 'label': 'swissgr...","[{'url': 'https://www.swissgrid.ch', 'label': ..."
3,energiedashboard.ch: Tägliche Flüsse in die un...,dataset,relations,4b5cdb6f-5dae-4704-9bce-3b2297ce0b26,"[{'url': 'https://transparency.entsoe.eu/', 'l...","[{'url': 'https://transparency.entsoe.eu/', 'l..."
4,energiedashboard.ch: Stromverbrauch Swissgrid,dataset,relations,0b9b12cc-8fad-47b2-a6fa-1fdd7f6a20a4,"[{'url': 'www.swissgrid.ch', 'label': 'swissgr...","[{'url': 'https://www.swissgrid.ch', 'label': ..."
5,"Füllungsgrad der Speicherseen, Sonntag 24h, Wo...",dataset,url,5ad2f1c1-12ec-440c-8ab2-b51f44ba3883,http://www.bfe.admin.ch/ogd17,https://www.bfe.admin.ch/ogd17
6,Schweizerische Elektrizitätsstatistik: Schweiz...,dataset,url,df79bf66-f51d-407e-a63b-331d4b516d8f,http://www.bfe.admin.ch/ogd35,https://www.bfe.admin.ch/ogd35
7,Solarenergiepotenziale der Schweizer Gemeinden,dataset,url,8c6a10fb-72f9-4f3c-bff5-e87d120bbd01,http://www.bfe.admin.ch/ogd52,https://www.bfe.admin.ch/ogd52
8,"Referenz-Marktpreise gemäss Art. 15, EnFV",dataset,url,eeadc4fb-8f11-4d13-a83b-4a0a0c816aec,http://www.bfe.admin.ch/kev,https://www.bfe.admin.ch/kev
9,Wochenstatistik Elektrizitätsbilanz - Erzeugun...,dataset,url,9ab8ccdd-acfd-442e-b315-8a44ff2536cb,http://www.bfe.admin.ch/ogd51,https://www.bfe.admin.ch/ogd51


In [26]:
f.to_csv('urlAndRelations_PROD.csv', index=False)
#f.to_csv('urlAndRelations_INT.csv', index=False)

-------------------------------------------------------------------------------------------------------------------------------------------------------------

For _resources_

In [27]:
# import module
from urllib.request import urlopen
from urllib.error import *
 
# try block to read URL

def checkLink(link):

  try:
      html = urlopen(link)
      
  # except block to catch
  # exception
  # and identify error
  except HTTPError as e:
      print("HTTP error", e)
      print(link)
      print()
      return False
      
  except URLError as e:
      print("Opps ! Page not found!", e)
      print(link)
      print()
      return False
  
  else:
      return True

In [28]:
def getURLs(original_string, urls):

    print(urls)
    
    links = []

    badLinks       = []

    adjustedString = '' 
    for s in original_string:
      adjustedString = adjustedString + s


    needsToBeListed = False

    
    for url in urls:
      
      url_modified    = ''
      # only store links which are faulty or need a protocol update (https)
      
      
      # if https already used
      if 'https' in url:
        working = checkLink(url)
        
        if not working:
          badLinks.append([url, '-'])
          #needsToBeListed = True
          
      # if https not in link
      else:
        
        url_modified = addHTTPS(url)
        working = checkLink(url_modified)
        
        if working:
          needsToBeListed = True
          # replace in adjusted_string
          adjustedString = adjustedString.replace(url, url_modified)
          
        else:
          # does not change http: urls
          url_modified = addHTTP(url)
          working = checkLink(url)
          
          if not working:
            badLinks.append([url, '-'])
                    
        
    if needsToBeListed:
      links.append([original_string, adjustedString])

    if len(badLinks) != 0:
      links = links + badLinks


    

    return links

In [29]:
def checkDictionary(l):

  resourceList = []
  
  for d in l:
    
    id = d['id']

    for key in d.keys():

      #print(d)

      value    = d[key]
      isString = True

      if type(value) != str:
        isString = False
        value    = str(d[key])


      urls = extractor.find_urls(value)
  
      # first remove elements which are not actual links
      urls = [url for url in urls if ('http' in url) or ('www' in url)]
      # do it only once if the url appears more than once
      urls = list(set(urls))

      whatToChange = []

      if len(urls) != 0:
        whatToChange = getURLs(value, urls)
      
      for elem in whatToChange:
        resourceList.append([key] + [id] + elem)

  return resourceList

In [30]:
#d = df.iloc[40]['resources'].copy()
#res = checkDictionary(d)

In [31]:
#for r in res:
#  print(r)

In [32]:
completeList = []

for index, row in df.iterrows():
  
  print(index)
  dataset_name = row['title']['de']
    
  att_type = 'resources'
  val      = row[att_type]

  liste = checkDictionary(val)
    
  if len(liste) != 0:
    for elem in liste:
      completeList.append([dataset_name] + [att_type] + elem)


0
['https://data.geo.admin.ch/ch.bfe.ladestellen-elektromobilitaet/data/oicp/ch.bfe.ladestellen-elektromobilitaet.json']
["https://github.com/SFOE/DIEMO-Documentation',"]
HTTP error HTTP Error 404: Not Found
https://github.com/SFOE/DIEMO-Documentation',

['https://data.geo.admin.ch/ch.bfe.ladestellen-elektromobilitaet/data/oicp/ch.bfe.ladestellen-elektromobilitaet.json']
['https://data.geo.admin.ch/ch.bfe.ladestellen-elektromobilitaet/status/oicp/ch.bfe.ladestellen-elektromobilitaet.json']
["https://github.com/SFOE/DIEMO-Documentation',"]
HTTP error HTTP Error 404: Not Found
https://github.com/SFOE/DIEMO-Documentation',

['https://data.geo.admin.ch/ch.bfe.ladestellen-elektromobilitaet/status/oicp/ch.bfe.ladestellen-elektromobilitaet.json']
1
['https://bfe-energy-dashboard-ogd.s3.amazonaws.com/ogd110_strom_verbrauch_prognose.csv']
['https://bfe-energy-dashboard-ogd.s3.amazonaws.com/ogd110_strom_verbrauch_prognose.csv']
2
['https://bfe-energy-dashboard-ogd.s3.amazonaws.com/ogd101_gas_imp

In [33]:
completeList

[['Ladestationen für Elektroautos',
  'resources',
  'description',
  'e33957be-180a-422b-90a5-fbfe9774927a',
  "https://github.com/SFOE/DIEMO-Documentation',",
  '-'],
 ['Ladestationen für Elektroautos',
  'resources',
  'description',
  '4d467a51-0bc9-48ce-aa2a-3d3bcaa7e7e9',
  "https://github.com/SFOE/DIEMO-Documentation',",
  '-'],
 ['Füllungsgrad der Speicherseen, Sonntag 24h, Wochenbericht Speicherinhalt',
  'resources',
  'url',
  'a2c8d73d-07b3-4e06-81f0-53b0b019b64b',
  'http://www.bfe.admin.ch/ogd17',
  'https://www.bfe.admin.ch/ogd17'],
 ['Thermische Netze (Nahwärme, Fernwärme, Fernkälte)',
  'resources',
  'download_url',
  'e85f0e09-3f51-491f-867e-34ae69f87b55',
  'http://data.geo.admin.ch/ch.bfe.thermische-netze/xtf/2056/ch.bfe.thermische-netze.zip',
  'https://data.geo.admin.ch/ch.bfe.thermische-netze/xtf/2056/ch.bfe.thermische-netze.zip'],
 ['Thermische Netze (Nahwärme, Fernwärme, Fernkälte)',
  'resources',
  'url',
  'e85f0e09-3f51-491f-867e-34ae69f87b55',
  'http://d

In [34]:
f = pd.DataFrame(completeList)
f.columns=['title', 'type', 'attribute', 'id', 'old_link', 'new_link']
f

,title,type,attribute,id,old_link,new_link
0,Ladestationen für Elektroautos,resources,description,e33957be-180a-422b-90a5-fbfe9774927a,"https://github.com/SFOE/DIEMO-Documentation',",-
1,Ladestationen für Elektroautos,resources,description,4d467a51-0bc9-48ce-aa2a-3d3bcaa7e7e9,"https://github.com/SFOE/DIEMO-Documentation',",-
2,"Füllungsgrad der Speicherseen, Sonntag 24h, Wo...",resources,url,a2c8d73d-07b3-4e06-81f0-53b0b019b64b,http://www.bfe.admin.ch/ogd17,https://www.bfe.admin.ch/ogd17
3,"Thermische Netze (Nahwärme, Fernwärme, Fernkälte)",resources,download_url,e85f0e09-3f51-491f-867e-34ae69f87b55,http://data.geo.admin.ch/ch.bfe.thermische-net...,https://data.geo.admin.ch/ch.bfe.thermische-ne...
4,"Thermische Netze (Nahwärme, Fernwärme, Fernkälte)",resources,url,e85f0e09-3f51-491f-867e-34ae69f87b55,http://data.geo.admin.ch/ch.bfe.thermische-net...,https://data.geo.admin.ch/ch.bfe.thermische-ne...
5,Projekte der Koordinationsstelle für nachhalti...,resources,download_url,9d80de78-9e24-4249-9aa1-eb383d2a11d9,http://data.geo.admin.ch/ch.bfe.komo-projekte/...,https://data.geo.admin.ch/ch.bfe.komo-projekte...
6,Projekte der Koordinationsstelle für nachhalti...,resources,url,9d80de78-9e24-4249-9aa1-eb383d2a11d9,http://data.geo.admin.ch/ch.bfe.komo-projekte/...,https://data.geo.admin.ch/ch.bfe.komo-projekte...
7,Projekte der Koordinationsstelle für nachhalti...,resources,download_url,2dfb89c8-4a5b-4482-963e-c984fa7e63b0,http://data.geo.admin.ch/ch.bfe.komo-projekte/...,https://data.geo.admin.ch/ch.bfe.komo-projekte...
8,Projekte der Koordinationsstelle für nachhalti...,resources,url,2dfb89c8-4a5b-4482-963e-c984fa7e63b0,http://data.geo.admin.ch/ch.bfe.komo-projekte/...,https://data.geo.admin.ch/ch.bfe.komo-projekte...
9,Schweizer Kernmaterialbestände im Ausland,resources,description,ef36ac3d-582a-483b-8c20-b6a5787d8c85,"https://frictionlessdata.io/',",-


In [35]:
f.to_csv('resources_PROD.csv', index=False)

In [36]:
df1 = pd.read_csv('urlAndRelations_PROD.csv')
df2 = pd.read_csv('resources_PROD.csv')

print(len(df1))
print(len(df2))

df = pd.concat([df1, df2], ignore_index=True)
df.sort_values(by=['title', 'type', 'attribute'], inplace=True, ignore_index=True)
df

40
37


,title,type,attribute,id,old_link,new_link
0,Analyse des schweizerischen Energieverbrauchs ...,dataset,url,4c963c00-a88e-48f4-98ef-d054af455c2f,http://www.bfe.admin.ch/ogd3,https://www.bfe.admin.ch/ogd3
1,Angebote der Innovationsförderung im Energiebe...,dataset,url,1d38a201-8edf-451b-a287-f734e7bc7a07,http://www.bfe.admin.ch/ogd4,https://www.bfe.admin.ch/ogd4
2,Beteiligung des BFE an internationalen Energie...,dataset,url,50e09d17-c87f-4cb5-9f7b-2803210a0343,http://www.bfe.admin.ch/ogd5,https://www.bfe.admin.ch/ogd5
3,Energieforschungsstatistik (EFstat),dataset,url,fe606459-cccb-4962-80ab-8d6697ad8401,http://www.bfe.admin.ch/ogd10,https://www.bfe.admin.ch/ogd10
4,Energieforschungsstatistik (EFstat),resources,url,1f8e2d1f-eb07-4e41-bd47-fca49d982025,https://lindas.admin.ch/sfoe/cube,-
...,...,...,...,...,...,...
72,Wochenstatistik Elektrizitätsbilanz - Erzeugun...,resources,uri,c4829b06-4dc8-4078-8cb4-a4f46b9e3282,https://ckan.opendata.swiss/dataset/9ab8ccdd-a...,-
73,energiedashboard.ch: Stromproduktion Swissgrid,dataset,relations,7676b090-6dff-483f-9d0c-166a0060b9b3,"[{'url': 'www.swissgrid.ch', 'label': 'swissgr...","[{'url': 'https://www.swissgrid.ch', 'label': ..."
74,energiedashboard.ch: Stromverbrauch Swissgrid,dataset,relations,0b9b12cc-8fad-47b2-a6fa-1fdd7f6a20a4,"[{'url': 'www.swissgrid.ch', 'label': 'swissgr...","[{'url': 'https://www.swissgrid.ch', 'label': ..."
75,energiedashboard.ch: Tägliche Flüsse in die un...,dataset,relations,6f77c60b-fd9a-44ac-914f-be3ccd4db5af,"[{'url': 'www.entsog.eu', 'label': 'ENTSO-G'},...","[{'url': 'https://www.entsog.eu', 'label': 'EN..."


--------------------------------------------------------------------------------------------------------------

Removing working links which have a strange element in the end

In [37]:
notWorking = df[df.new_link == '-'].copy()

In [40]:
to_change = []

for index, row in notWorking.iterrows():
  link = row['old_link']
  lastElem = link[-1]

  if len(re.findall("\W", lastElem)) != 0 and lastElem != '/':
    print(index)

    while len(re.findall("\W", lastElem)) != 0:
      link = link[:len(link)-1]
      lastElem = link[-1]

    #working = checkLink(link)
    working = check_url(link)

    to_change.append([index, working, link])

18
21
22
33
44
45
57
58


In [41]:
for elem in to_change:
  if elem[1]:
    df.drop(elem[0], inplace=True)
  else:
    df.at[elem[0], 'old_link'] = elem[3]

In [42]:
df.reset_index(drop = True, inplace=True)

In [43]:
#df.to_csv('INT_linksToChange.csv', index=False)
#df.to_json('INT_linksToChange.json', orient='records')

df.to_csv('PROD_linksToChange.csv', index=False)
df.to_json('PROD_linksToChange.json', orient='records')